In [1]:
import pandas as pd
from sklearn.metrics import root_mean_squared_error, precision_score, recall_score, accuracy_score

In [3]:
processed_ratings_file = "./Data/rating_test_df_u4.csv"
rating_test_df = pd.read_csv(processed_ratings_file)

cleaned_df = rating_test_df.dropna(subset=['simulated_ratings'])
cleaned_df.head(100)

,Unnamed: 0,user_id,movie_id,rating,timestamp,simulated_ratings
0,0,1,4,3,876893119,4.0
1,1,1,7,4,875071561,2.0
2,2,1,42,5,876892425,3.0
3,3,1,43,4,878542869,3.0
4,4,1,55,5,875072688,4.0
...,...,...,...,...,...,...
95,97,8,431,2,879362356,4.0
96,98,8,510,4,879362233,4.0
97,99,9,50,5,886960055,4.0
98,100,9,201,5,886960055,2.0


In [4]:

# Ensure the values are integers
cleaned_df['rating'] = cleaned_df['rating'].astype(int)
cleaned_df['simulated_ratings'] = cleaned_df['simulated_ratings'].astype(int)

# RMSE
rmse = root_mean_squared_error(cleaned_df['rating'], cleaned_df['simulated_ratings'])

# Exact match
exact_matches = (cleaned_df['rating'] == cleaned_df['simulated_ratings'])
exact_match_count = exact_matches.sum()
exact_match_percentage = exact_match_count / len(cleaned_df) * 100

# Off by 1 level
off_by_1 = (cleaned_df['rating'] - cleaned_df['simulated_ratings']).abs() == 1
off_by_1_count = off_by_1.sum()
off_by_1_percentage = off_by_1_count / len(cleaned_df) * 100

# Off by more than 1 level
off_by_more_than_1 = (cleaned_df['rating'] - cleaned_df['simulated_ratings']).abs() > 1
off_by_more_than_1_count = off_by_more_than_1.sum()
off_by_more_than_1_percentage = off_by_more_than_1_count / len(cleaned_df) * 100

# Weighted accuracy
weighted_accuracy = (exact_matches * 1 + off_by_1 * 0.8).sum() / len(cleaned_df)

# Output the results
print(f"RMSE: {rmse}")
print(f"Exact match count: {exact_match_count}")
print(f"Exact match percentage: {exact_match_percentage:.2f}%")
print(f"Off by 1 level count: {off_by_1_count}")
print(f"Off by 1 level percentage: {off_by_1_percentage:.2f}%")
print(f"Off by more than 1 level count: {off_by_more_than_1_count}")
print(f"Off by more than 1 level percentage: {off_by_more_than_1_percentage:.2f}%")
print(f"Weighted Accuracy: {weighted_accuracy:.2f}")


RMSE: 1.308348623619631
Exact match count: 6298
Exact match percentage: 32.16%
Off by 1 level count: 8702
Off by 1 level percentage: 44.44%
Off by more than 1 level count: 4582
Off by more than 1 level percentage: 23.40%
Weighted Accuracy: 0.68


C:\Users\dania\AppData\Local\Temp\ipykernel_5256\1580610837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['rating'] = cleaned_df['rating'].astype(int)
C:\Users\dania\AppData\Local\Temp\ipykernel_5256\1580610837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['simulated_ratings'] = cleaned_df['simulated_ratings'].astype(int)
